# Virtual flight over image

In [1]:
from moviepy.editor import ImageClip, concatenate
import moviepy.video.fx.all as vfx

from skimage import transform
import numpy as np

In [2]:
 # function borrowed from https://zulko.github.io/moviepy/examples/star_worms.html
def trapzWarp(pic,cx,cy):
    """ Complicated function (will be latex packaged as a fx) """
    Y,X = pic.shape[:2]
    src = np.array([[0,0],[X,0],[X,Y],[0,Y]])
    dst = np.array([[cx*X,cy*Y],[(1-cx)*X,cy*Y],[X,Y],[0,Y]])
    tform = transform.ProjectiveTransform()
    tform.estimate(src,dst)
    im = transform.warp(pic, tform.inverse, output_shape=(Y,X))
    return (im*255).astype('uint8')

In [3]:
def fly(image_paths, video_res=(1920, 1080), duration=20, perspective=0.15,
        labels=None, text_color='white', fontsize=32, 
        add_logos=True):
    
    """Creates a virtual flight video from an image file

    Parameters
    ----------
    image_path : str
        Path to the image file
    video_res: (int, int)
        Resolution (width, height) of the output video (default: (1920, 1080))
            wider video -> faster scrolling
            higher video -> slower scrolling
    duration : int
        Duration of the video in seconds: (default: 5)
            higher -> longer videos -> slower flight
            lower -> shorter videos -> faster flight
    perspective : float
        Perspective parameter (default: 0.2)
            0.0 -> No perspective transformation
            smaller -> less perspective transformation -> slower flight
            higher (max = 0.499) -> more perspective transformation -> faster flight
        
    Returns
    -------
    final_with_text
        A moviepy video

    """
    
    labels.reverse()
    image_paths.reverse()
    
    repeating_list = []
    rep_text_list = []
    iterations = int(duration / len(image_paths))
    for x in range(0, iterations):
        repeating_list.extend(image_paths)
        rep_text_list.extend(labels)

    img_clips = []
    txt_clips = []
    for text, image_path in zip(rep_text_list, repeating_list):
        
        clip_duration = duration / len(repeating_list)
        
        img_clip = ImageClip(image_path).set_duration(clip_duration)
        img_clips.append(img_clip)
        
        txt_clip = TextClip(text, 
                            fontsize = fontsize, 
                            color = text_color).set_duration(clip_duration)
        txt_clips.append(txt_clip)

    txt_video = concatenate(txt_clips, method="compose")
    full_video = concatenate(img_clips, method="compose")
    
    out_width, out_height = video_res
    
    # Resize for correct width after cropping
    width = out_width / (1 - 2 * perspective)
    resized = full_video.resize(width=width)
    
    # Scroll backwards
    pixel_to_scroll = resized.h - out_height
    speed = pixel_to_scroll / duration # pixel per second
    scrolling = vfx.scroll(resized, h=out_height, y_speed=speed)
    backwards = vfx.time_mirror(scrolling)
    txt_video = vfx.time_mirror(txt_video)
    
    # Add perspective
    fl_im = lambda pic : trapzWarp(pic, perspective, 0.0)
    warped = backwards.fl_image(fl_im)
    
    # Clip sides width no data
    clipping = width * perspective
    cropped = vfx.crop(warped, x1=clipping, width=out_width)
    
    # Add Logos
    if add_logos:
        logo_left = (mp.ImageClip("data/logo_left_2.png")
              .set_duration(cropped.duration)
              .resize(height=int(cropped.h * 0.05))
              .margin(left=8, bottom=8, opacity=0)
              .set_pos(("left","bottom")))

        logo_right = (mp.ImageClip("data/logo_right_2.png")
              .set_duration(cropped.duration)
              .resize(height=int(cropped.h * 0.05))
              .margin(right=8, bottom=8, opacity=0)
              .set_pos(("right","bottom")))
        
        final = mp.CompositeVideoClip([cropped, logo_left, logo_right])
    else:
        final = cropped
        
    final_with_text = CompositeVideoClip([final, txt_video.set_position((8,8))]) 

    return final_with_text

In [ ]:
image_paths = ['image_1.tif', 'optional_image_2.tif']
mp4_path = 'video.mp4'

In [ ]:
fly(image_paths, video_res=(1280, 720), duration=20, perspective=0.15, labels=labels).write_videofile(str(mp4_path), fps=30)